In [59]:
import requests
import pandas as pd
import numpy as np
import requests
import json
import urllib
from collections import defaultdict
from operator import itemgetter

In [ ]:
import os

home_relative_dir = '..'

os.chdir(home_relative_dir)
os.getcwd()

In [2]:
with open('apikey.txt','r') as f:
    api_key = f.read().strip()

In [45]:
# routes
r = requests.get(f'http://bustime.mta.info/api/where/routes-for-agency/MTA%20NYCT.json',params=[('key',api_key)])
all_routes = json.loads(r.content)

In [70]:
def get_vehicle_data():
    r = requests.get('http://bustime.mta.info/api/siri/vehicle-monitoring.json',params=[('key',api_key),('version',2)])
    if r.ok:
        return json.loads(r.content)['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
    print('Vehicle GET call failed')
    return None

In [71]:
vehicle_activities = get_vehicle_data()

In [74]:
def extract_groups(vehicle_activities):
    groups = defaultdict(lambda : defaultdict(list))
    for vehicle_activity in vehicle_activities:
        vehicle_activity = vehicle_activity['MonitoredVehicleJourney']
        line_ref,direction_ref, location,v_id = itemgetter('LineRef', 'DirectionRef','VehicleLocation','VehicleRef')(vehicle_activity)
        groups[line_ref][direction_ref].append({
            'vehicle_id':v_id,
            'location':location
        })
    return groups

In [73]:
groups

defaultdict(<function __main__.<lambda>()>,
            {'MTA NYCT_M116': defaultdict(list,
                         {'1': [{'vehicle_id': 'MTA NYCT_4082',
                            'location': {'Longitude': -73.945488,
                             'Latitude': 40.800248}},
                           {'vehicle_id': 'MTA NYCT_3873',
                            'location': {'Longitude': -73.959968,
                             'Latitude': 40.800706}}],
                          '0': [{'vehicle_id': 'MTA NYCT_9777',
                            'location': {'Longitude': -73.962553,
                             'Latitude': 40.799105}},
                           {'vehicle_id': 'MTA NYCT_4015',
                            'location': {'Longitude': -73.931061,
                             'Latitude': 40.797544}},
                           {'vehicle_id': 'MTA NYCT_9734',
                            'location': {'Longitude': -73.930664,
                             'Latitude': 40.797375}},
  

In [82]:
from haversine import Unit
import haversine as hs
loc1=(28.426846,77.088834)
loc2=(28.394231,77.050308)
hs.haversine(loc1,loc2,unit=Unit.MILES)

3.2495929643035977

In [94]:
def calculate_distance(loc1,loc2,unit = Unit.MILES):
    return hs.haversine((loc1['Latitude'],loc1['Longitude']),(loc2['Latitude'],loc2['Longitude']),unit=unit)

In [97]:
THRESH = 0.3

In [100]:
for route,route_vehicles in groups.items():
    for route_dir,dir_vehicles in route_vehicles.items():
        l = len(dir_vehicles)
        for i in range(l):
            for j in range(i+1,l):
                distance = calculate_distance(dir_vehicles[i]['location'],dir_vehicles[j]['location'])
                if distance<THRESH:
                    print(f'{dir_vehicles[i]["vehicle_id"]} and {dir_vehicles[i]["vehicle_id"]} are too close.')

MTA NYCT_4015 and MTA NYCT_4015 are too close.
MTA NYCT_9421 and MTA NYCT_9421 are too close.
MTA NYCT_9774 and MTA NYCT_9774 are too close.
MTA NYCT_9732 and MTA NYCT_9732 are too close.
MTA NYCT_9756 and MTA NYCT_9756 are too close.
MTA NYCT_4693 and MTA NYCT_4693 are too close.
MTA NYCT_9875 and MTA NYCT_9875 are too close.
MTA NYCT_4018 and MTA NYCT_4018 are too close.
MTA NYCT_9721 and MTA NYCT_9721 are too close.
MTA NYCT_9737 and MTA NYCT_9737 are too close.
MTA NYCT_3943 and MTA NYCT_3943 are too close.
MTA NYCT_9749 and MTA NYCT_9749 are too close.
MTA NYCT_7113 and MTA NYCT_7113 are too close.
MTA NYCT_7113 and MTA NYCT_7113 are too close.
MTA NYCT_7116 and MTA NYCT_7116 are too close.
MTA NYCT_4832 and MTA NYCT_4832 are too close.
MTA NYCT_7117 and MTA NYCT_7117 are too close.
MTA NYCT_7128 and MTA NYCT_7128 are too close.
MTA NYCT_4819 and MTA NYCT_4819 are too close.
MTA NYCT_8441 and MTA NYCT_8441 are too close.
MTA NYCT_8450 and MTA NYCT_8450 are too close.
MTA NYCT_7548

In [102]:
os.getcwd()

'/Users/adityachawla/skunkworks22/RightRoute'

In [103]:
os.listdir()

['frontend',
 'backend',
 'firestore-access-key.json',
 'apikey.txt',
 '.gitignore',
 'scripts',
 '.git']

In [111]:
path_to_service_acc = '/Users/adityachawla/skunkworks22/RightRoute/firestore-access-key.json'

In [112]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use the application default credentials
cred = credentials.ApplicationDefault()
project_id = 'aditya-gcp-342814'
cred = credentials.Certificate(path_to_service_acc)
firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.